In [42]:
import math
from scipy.stats import norm

---
## Задание №1

__Рассчитайте выборку (с учётом dropout rate)__ для следующей конечной точки:
> Доля пациентов, которые достигли клинического ответа на терапию на визите 6 (уменьшение интенсивности боли минимум на 3 балла)

__Также у вас есть дополнительные ограничения на расчёт выборки:__
1. Вы хотите сравнить препарат "TheNew" (новый препарат) с препаратом "TheGold" (уже на рынке)
2. Тестировать будете гипотезу превосходства
3. Собираетесь в тестовую группу включить столько же людей, сколько и в группу сравнения
4. Ошибки первого и второго рода для одной точки следующие:
    $$\alpha=0.05, \beta=0.2 $$
5. Ожидаемая доля ответчиков в группе "TheNew" - 0.65, а в группе "TheGold" - 0.53
6. Вы собираетесь проанализировать данные при помощи критерия хи-квадрат
7. Предполагаемый dropout rate - 10%
---
__Примечание__. Возможно, стоит посмотреть следующую (скорректированную) формулу для расчёта выборки: https://stats.stackexchange.com/questions/340291/estimate-sample-size-for-chi-squared-test

Более подробно о ней можно прочитать здесь (раздел 4.1): https://books.google.com/books?id=9VefO7a8GeAC&pg=PT1&hl=ru&source=gbs_selected_pages&cad=3#v=onepage&q&f=false

Зададим функцию для расчёта выборки (без учёта dropout rate). Функция нам пригодится в задании №3.

In [43]:
def firstEndPoint(alpha, beta):
    # Доля ответчиков
    p1 = 0.65
    p2 = 0.53
    effect_size = abs(p2 - p1)

    # Рассчитываем необходимые параметры Z
    z_alpha = abs(norm.ppf(alpha / 2))
    z_beta = abs(norm.ppf(beta))

    # Рассчитываем выборку
    q1 = 1 - p1
    q2 = 1 - p2
    Q = (q1 + q2) / 2
    P = (p1 + p2) / 2
    ns = ((z_alpha * math.sqrt(2*P*Q) + z_beta * math.sqrt(p1*q1 + p2*q2)) ** 2) / (effect_size ** 2)
    return math.ceil((ns / 4) * (1 + math.sqrt(1 + (4 / (ns * effect_size)))))

In [44]:
# Задаем необходимые параметры alpha и beta
alpha1 = 0.05
beta1 = 0.2

# Вычислим размер выборки
n = firstEndPoint(alpha1, beta1)

# Расчитаем размер выборки с учётом dropout rate
dropout_rate = 0.1
n_drop_out = math.ceil(n/(1-dropout_rate))

# Выводим результат
print(f"Необходимое количество субъектов для исследования: {2*n_drop_out}")

Необходимое количество субъектов для исследования: 304


---
## Задание №2

__Рассчитайте выборку (с учётом dropout rate)__ для следующей конечной точки:
> Разница во времени возвращения боли между группами лечения, измеренная в соотнотношении уровней опасностей в каждой из групп

__Также у вас есть дополнительные ограничения на расчёт выборки:__
1. Вы хотите сравнить препарат "TheNew" (новый препарат) с препаратом "TheGold" (уже на рынке)
2. Тестировать будете гипотезу превосходства, причём вы хотите протестировать гипотезу **superiority by margin**, где margin = 0.1
3. Собираетесь в тестовую группу включить столько же людей, сколько и в группу сравнения
4. Ошибки первого и второго рода для одной точки следующие:
    $$\alpha=0.05, \beta=0.2 $$
5. Вы собираетесь проанализировать данные при помощи регресии Кокса, так как знаете, что на время возвращения боли влияет пол пациента и тип региона проживания (большой город и маленький) 

6. Вы предполагаете, что соотношение уровней опасности (Hazard Ratio, HR) следующее:
$$ HR = \frac{Hazard_1}{Hazard_2} = \frac{0.91}{0.7} = 1.3, $$ где Hazard_1 - уровень риска в группе TheGold, а Hazard_2 - уровень риска в группе TheNew

Вы изучили литературу, и поняли, что для расчёта хорошо подойдёт следующая формула:
$$
n = \frac{(Z_{\alpha/2}+Z_{\beta})^2}{(ln(HR)-\delta)^2p_1p_2}
$$

Здесь $$ \delta - это\ margin \newline p_1\ и\ p_2 — доля\ пациентов, рандомизированных\ для\ лечения\ TheGold\ и\ TheNew\ соответственно.$$
8. Предполагаемый dropout rate - 10%


---
Примечание: формула расчёта выборки для этого случая неплохо изложена в этом источнике на страницах 177-178: https://mobt3ath.com/uplode/book/book-43792.pdf

Зададим функцию для расчёта выборки (без учёта dropout rate). Функция нам пригодится в задании №3.

In [45]:
def secondEndPoint(alpha, beta):
    # Доля ответчиков
    p1 = 0.65
    p2 = 0.53

    # Зададим соотношение уроыней опасности
    Hazard_1 = 0.91
    Hazard_2 = 0.7
    HR = Hazard_1/Hazard_2

    # Мы хотим протестировать гипотезу superiority by margin
    margin = 0.1

    z_alpha = abs(norm.ppf(alpha / 2))
    z_beta = abs(norm.ppf(beta))

    return math.ceil(((z_alpha + z_beta) ** 2) / (((math.log(HR) - margin) ** 2) * p1 * p2))

In [46]:
# Задаем необходимые параметры alpha и beta
alpha2 = 0.05
beta2 = 0.2

# Вычислим размер выборки
n = secondEndPoint(alpha2, beta2)

# Расчитаем размер выборки с учётом dropout rate
dropout_rate = 0.1
n_drop_out = math.ceil(n/(1-dropout_rate))

# Выводим результат
print(f"Необходимое количество субъектов для исследования: {2*n_drop_out}")

Необходимое количество субъектов для исследования: 1924


---
## Задание №3

Теперь обе конечные точки - первичные конечные точки исследования, исходя из чего необходимо рассчитать выборку.

__Рассчитайте выборку (с учётом drop-out rate)__ для обеих конечных точек выше по следующим сценариям:
1. Успех исследования - обе конечные точки успешны
2. Успех исследования - хотя бы одна конечная точка успешна

Dropout rate = 10%

Рассчитаем размер выборки для сценария, где успехом исследования считается успех обоих конечных точек:

In [47]:
# Вычислим параметры alpha и beta
alpha_success_all = (alpha1 + alpha2)/2
beta_success_all = beta1

# Вычислим размер выборок для каждой конечной точки
n_success_all_1 = firstEndPoint(alpha_success_all, beta_success_all)
n_success_all_2 = secondEndPoint(alpha_success_all, beta_success_all)

# Найдем необходимый размер выборки:
qty1 = max(n_success_all_1, n_success_all_2)

# Рассчитаем выборку с учетом drop-out-rate
dropout_rate = 0.1
qty1_dropout_rate = math.ceil(qty1/(1-dropout_rate))

# Выводим результат
print('Необходимое количество субъектов для исследования:', qty1_dropout_rate)

Необходимое количество субъектов для исследования: 962


Рассчитаем размер выборки для сценария, где успехом исследования считается успех хотя бы одной конечной точки:

In [48]:
# Вычислим параметры alpha и beta
alpha_success_any = alpha1
beta_success_any = (1 - beta1)*(1 - beta2)

# Вычислим размер выборок для каждой конечной точки
n_success_any_1 = firstEndPoint(alpha_success_any, beta_success_any)
n_success_any_2 = secondEndPoint(alpha_success_any, beta_success_any)

# Найдем необходимый размер выборки:
qty2 = max(n_success_any_1, n_success_any_2)

# Рассчитаем выборку с учетом drop-out-rate
dropout_rate = 0.1
qty2_dropout_rate = math.ceil(qty2/(1-dropout_rate))

# Выводим результат
print('Необходимое количество субъектов для исследования:', qty2_dropout_rate)

Необходимое количество субъектов для исследования: 658
